In [13]:
import os
if "ntbk" in os.getcwd():
    os.chdir("..")
print(os.getcwd())

import sys
sys.path.append(os.path.join(os.getcwd(), "otgnn"))


%load_ext autoreload
%autoreload 2

/home/octav/gitrepos/tum-thesis
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
from otgnn.models import GCN
from otgnn.graph import MolGraph
from otgnn.utils import save_model, load_model

from mol_opt.data_mol_opt import MolOptDataset
from mol_opt.data_mol_opt import get_loader
from mol_opt.arguments import get_args
from mol_opt.train_mol_opt import main, get_latest_model

from rdkit.Chem import MolFromSmiles

import torch
import numpy as np
import time

In [15]:
os.getcwd()

'/home/octav/gitrepos/tum-thesis'

In [16]:
data_loader = get_loader("iclr19-graph2graph/data/qed", "val", 48, True)

In [17]:
for i in data_loader:
    X = (MolGraph(i[0]))
    Y = (MolGraph(i[1]))
    break

In [18]:
len(X.mols)

48

In [19]:
sys.argv = ["", "-cuda"]
args = get_args()
args.output_dir = "mol_opt/output/"
# args.device = "cpu"
gcn = GCN(args).to(args.device)
opt = torch.nn.Linear(50, 50).to(args.device)

In [20]:
gcn = GCN(args).to(args.device)

In [21]:
gcn

GCN(
  (W_message_i): Linear(in_features=100, out_features=50, bias=False)
  (W_message_h): Linear(in_features=50, out_features=50, bias=False)
  (W_message_o): Linear(in_features=143, out_features=50, bias=True)
  (W_mol_h): Linear(in_features=50, out_features=100, bias=True)
  (W_mol_o): Linear(in_features=100, out_features=1, bias=True)
  (dropout_gcn): Dropout(p=0.0, inplace=False)
  (dropout_ffn): Dropout(p=0.0, inplace=False)
)

In [22]:
gcn

GCN(
  (W_message_i): Linear(in_features=100, out_features=50, bias=False)
  (W_message_h): Linear(in_features=50, out_features=50, bias=False)
  (W_message_o): Linear(in_features=143, out_features=50, bias=True)
  (W_mol_h): Linear(in_features=50, out_features=100, bias=True)
  (W_mol_o): Linear(in_features=100, out_features=1, bias=True)
  (dropout_gcn): Dropout(p=0.0, inplace=False)
  (dropout_ffn): Dropout(p=0.0, inplace=False)
)

In [23]:
opt(gcn.forward(Y)[0])

tensor([[ 0.0992, -0.0890,  0.1723,  ...,  0.0831,  0.0952,  0.1011],
        [ 0.0569, -0.1528,  0.1374,  ...,  0.0197,  0.0662,  0.2205],
        [ 0.0497, -0.1381,  0.1233,  ...,  0.0302,  0.0377,  0.1758],
        ...,
        [ 0.0960, -0.0695,  0.1740,  ...,  0.0660,  0.1047,  0.0696],
        [ 0.0818, -0.1535,  0.1402,  ...,  0.0942,  0.1711,  0.2707],
        [-0.0520, -0.1821,  0.1257,  ...,  0.0616,  0.1846,  0.2143]],
       device='cuda:0', grad_fn=<AddmmBackward>)

In [24]:
gcn.forward(X)[0].shape

torch.Size([1061, 50])

In [25]:
print (np.cumsum(([len(x.atoms) for x in X.mols])))

[  18   42   61   87  106  130  150  167  187  212  237  262  286  308
  331  353  375  394  418  437  457  477  500  526  552  575  598  622
  643  663  688  711  735  757  778  799  816  842  862  884  905  929
  950  969  993 1017 1039 1061]


In [26]:
x = X.mols[0]

In [27]:
from mol_opt import mol_opt
molopt = mol_opt.MolOpt(args).to(device = args.device)
molopt

MolOpt(
  (GCN): GCN(
    (W_message_i): Linear(in_features=100, out_features=50, bias=False)
    (W_message_h): Linear(in_features=50, out_features=50, bias=False)
    (W_message_o): Linear(in_features=143, out_features=50, bias=True)
    (W_mol_h): Linear(in_features=50, out_features=100, bias=True)
    (W_mol_o): Linear(in_features=100, out_features=1, bias=True)
    (dropout_gcn): Dropout(p=0.0, inplace=False)
    (dropout_ffn): Dropout(p=0.0, inplace=False)
  )
  (opt0): Linear(in_features=50, out_features=50, bias=True)
  (opt1): Linear(in_features=50, out_features=50, bias=True)
  (delta_loss): MSELoss()
)

In [28]:
molopt.align(molopt.encode(X), X, molopt.encode(Y), Y)

tensor([[-0.3221,  0.0038,  0.0035,  ..., -0.1331,  0.1204,  0.1619],
        [-0.0960, -0.0575,  0.0851,  ..., -0.1013,  0.1716,  0.2112],
        [-0.1319,  0.0630, -0.1862,  ...,  0.0449,  0.1181,  0.1438],
        ...,
        [-0.1173, -0.0720,  0.1398,  ..., -0.0912,  0.1875,  0.2520],
        [-0.0846, -0.0290, -0.0092,  ..., -0.2125,  0.1703,  0.1214],
        [-0.1873,  0.0246, -0.1750,  ...,  0.0147,  0.1562,  0.2467]],
       device='cuda:0', grad_fn=<CopySlices>)

In [29]:
molopt.encode(Y)

tensor([[-0.3221,  0.0038,  0.0035,  ..., -0.1331,  0.1204,  0.1619],
        [-0.1142,  0.0280,  0.1557,  ..., -0.2182,  0.1711,  0.2604],
        [-0.1235, -0.0081,  0.0759,  ..., -0.1747,  0.1613,  0.2248],
        ...,
        [-0.2643,  0.0470, -0.0293,  ..., -0.1391,  0.1221,  0.1442],
        [-0.1873,  0.0246, -0.1750,  ...,  0.0147,  0.1562,  0.2467],
        [-0.0846, -0.0290, -0.0092,  ..., -0.2125,  0.1703,  0.1214]],
       device='cuda:0', grad_fn=<AddmmBackward>)

In [30]:
molopt.forward_train(X, Y)

tensor(0.0144, device='cuda:0', grad_fn=<MeanBackward0>)

In [31]:
save_model(molopt, args, args.output_dir, "test")

Model saved to: mol_opt/output//model_test


In [32]:
args.output_dir

'mol_opt/output/'

In [33]:
# for epoch in range(args.n_epochs):
#     start = time.time()
#     for idx, i in enumerate(data_loader):
#         X = (MolGraph(i[0]))
#         Y = (MolGraph(i[1]))
# 
#         # create your optimizer
#         optimizer = torch.optim.SGD(molopt.parameters(), lr=0.01)
# 
#         # in your training loop:
#         optimizer.zero_grad()   # zero the gradient buffers
#         loss = molopt.forward_train(X, Y)
#         loss.backward()
#         optimizer.step()    # Does the update
# 
#         print ("Iter: {}, loss: {}".format(idx, loss.item()))
#     end = time.time()
#     print("Time for epoch {}: {}", epoch, end - start)

In [34]:
X.scope

[(0, 18),
 (18, 24),
 (42, 19),
 (61, 26),
 (87, 19),
 (106, 24),
 (130, 20),
 (150, 17),
 (167, 20),
 (187, 25),
 (212, 25),
 (237, 25),
 (262, 24),
 (286, 22),
 (308, 23),
 (331, 22),
 (353, 22),
 (375, 19),
 (394, 24),
 (418, 19),
 (437, 20),
 (457, 20),
 (477, 23),
 (500, 26),
 (526, 26),
 (552, 23),
 (575, 23),
 (598, 24),
 (622, 21),
 (643, 20),
 (663, 25),
 (688, 23),
 (711, 24),
 (735, 22),
 (757, 21),
 (778, 21),
 (799, 17),
 (816, 26),
 (842, 20),
 (862, 22),
 (884, 21),
 (905, 24),
 (929, 21),
 (950, 19),
 (969, 24),
 (993, 24),
 (1017, 22),
 (1039, 22)]

In [138]:
x_embedding.narrow(0, 0, 22)

tensor([[-0.0455,  0.0447,  0.0261,  ..., -0.0160,  0.0526,  0.0204],
        [-0.0465,  0.0597,  0.0177,  ..., -0.1785, -0.0618,  0.0922],
        [ 0.0161,  0.0673, -0.1091,  ...,  0.0081, -0.0677,  0.1738],
        ...,
        [ 0.0850, -0.0669, -0.0290,  ..., -0.0392, -0.0300,  0.0311],
        [ 0.1073, -0.0826, -0.0462,  ..., -0.0227, -0.0654,  0.0300],
        [ 0.0796,  0.0683,  0.0362,  ..., -0.0448, -0.0863,  0.0600]],
       device='cuda:0', grad_fn=<SliceBackward>)

In [144]:
x_embedding[0:22,:]

torch.Size([22, 50])

In [136]:
x_embedding.narrow(0, 22, 18)

tensor([[-9.2475e-02,  4.4592e-02,  4.1157e-02, -6.2958e-02, -2.6449e-02,
          5.1672e-03, -4.6384e-02, -5.3797e-02, -7.3464e-02, -7.3005e-02,
         -1.7704e-02,  8.3490e-02, -1.3573e-01,  8.2092e-03, -5.4852e-02,
         -1.0005e-01,  1.0265e-01,  8.8467e-02,  1.8191e-02,  8.6351e-02,
          1.7423e-01,  2.4751e-02,  6.3913e-03,  3.5624e-03,  8.4073e-03,
         -2.3538e-02,  1.3579e-01,  4.8769e-02,  1.5200e-01, -2.5218e-02,
          2.4999e-01, -7.4404e-03,  6.2257e-02, -1.2847e-01, -1.4130e-01,
          1.4734e-01, -2.1842e-03,  8.3792e-02, -8.7801e-02,  4.2352e-02,
          2.5093e-01, -3.2312e-02,  7.7363e-02,  6.2747e-02,  1.1126e-01,
          6.4715e-02, -3.6868e-02,  1.1756e-03,  7.7242e-03,  7.6595e-03],
        [ 2.0119e-03, -1.1420e-01, -6.6094e-03, -6.3875e-02,  8.7873e-02,
          1.7288e-02,  9.8871e-02, -1.4052e-01, -5.1341e-02, -1.6861e-01,
         -7.1023e-02,  3.3994e-02,  2.1864e-02, -9.5770e-03,  4.5321e-02,
          1.1468e-01,  1.7144e-01,  1

In [35]:
# start = time.time()
# for idx, i in enumerate(data_loader):
#     X = (MolGraph(i[0]))
#     Y = (MolGraph(i[1]))
#     
#     # create your optimizer
#     optimizer = torch.optim.SGD(molopt.parameters(), lr=0.01)
# 
#     # in your training loop:
#     optimizer.zero_grad()   # zero the gradient buffers
#     loss = molopt.forward_train(X, Y)
#     loss.backward()
#     optimizer.step()    # Does the update
# 
#     print (idx, loss)
# end = time.time()
# print(end - start)

# run the implemented function

In [50]:
args = get_args()
args.n_epochs = 200 
args.output_dir = "mol_opt/output"
args.init_model = "gcn2"
args

Namespace(agg_func='sum', batch_norm=False, cuda=True, device='cuda:0', dropout_ffn=0.0, dropout_gcn=0.0, ffn_activation='LeakyReLU', init_model='gcn2', linear_out=False, n_epochs=200, n_ffn_hidden=100, n_hidden=50, n_labels=1, n_layers=5, output_dir='mol_opt/output', pc_hidden=50)

In [39]:
train_data_loader = get_loader("iclr19-graph2graph/data/qed", "train", 48, True)
val_data_loader = get_loader("iclr19-graph2graph/data/qed", "val", 48, True)

In [46]:
get_latest_model("gcn2", args.output_dir)

No model gcn2 found in mol_opt/output! Starting from scratch.


(None, 0)

In [53]:
molopt = main(args, train_data_loader = train_data_loader, val_data_loader = val_data_loader)

Epoch: 30
 train_mse: 0.000310
 val_mse: 0.000002
Epoch duration: 27.781742334365845
Model saved to: mol_opt/output/model_gcn2_30
Epoch: 31
 train_mse: 0.000000
 val_mse: 0.000001
Epoch duration: 27.766279697418213
Model saved to: mol_opt/output/model_gcn2_31
Epoch: 32
 train_mse: 0.000000
 val_mse: 0.000000
Epoch duration: 28.335959672927856
Model saved to: mol_opt/output/model_gcn2_32
Epoch: 33
 train_mse: 0.000000
 val_mse: 0.000000
Epoch duration: 28.279423713684082
Model saved to: mol_opt/output/model_gcn2_33
Epoch: 34
 train_mse: 0.000000
 val_mse: 0.000000
Epoch duration: 27.708338737487793
Model saved to: mol_opt/output/model_gcn2_34
Epoch: 35
 train_mse: 0.000000


Process Process-89:
Traceback (most recent call last):
  File "/home/octav/anaconda3/envs/mol_ot/lib/python3.6/threading.py", line 551, in wait
    signaled = self._cond.wait(timeout)
  File "/home/octav/anaconda3/envs/mol_ot/lib/python3.6/threading.py", line 304, in wait
    self._acquire_restore(saved_state)
  File "/home/octav/anaconda3/envs/mol_ot/lib/python3.6/threading.py", line 251, in _acquire_restore
    def _acquire_restore(self, x):
RuntimeError: release unlocked lock
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):


KeyboardInterrupt: 

  File "/home/octav/anaconda3/envs/mol_ot/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/octav/anaconda3/envs/mol_ot/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/octav/anaconda3/envs/mol_ot/lib/python3.6/site-packages/torch/utils/data/_utils/worker.py", line 192, in _worker_loop
    data_queue.put((idx, data))
  File "/home/octav/anaconda3/envs/mol_ot/lib/python3.6/multiprocessing/queues.py", line 87, in put
    self._start_thread()
  File "/home/octav/anaconda3/envs/mol_ot/lib/python3.6/multiprocessing/queues.py", line 169, in _start_thread
    self._thread.start()
  File "/home/octav/anaconda3/envs/mol_ot/lib/python3.6/threading.py", line 851, in start
    self._started.wait()
  File "/home/octav/anaconda3/envs/mol_ot/lib/python3.6/threading.py", line 552, in wait
    return signaled
  File "/home/octav/anaconda3/envs/mol_ot/lib/python3.6/threading.py", line 243,

# try out some results

In [164]:
for i in data_loader:
    X = (MolGraph(i[0]))
    Y = (MolGraph(i[1]))
    break

In [178]:
x_embedding, x_delta = molopt.forward(X)
y_embedding = molopt.encode(Y)
y_aligned = molopt.align(x_embedding, X, y_embedding, Y)
xhat_delta = molopt.delta(x_embedding, y_aligned)

In [172]:
x_embedding.shape

torch.Size([1041, 50])

In [173]:
x_delta

tensor([[-0.0329,  0.0609, -0.0998,  ...,  0.0075,  0.0955,  0.0417],
        [-0.0989,  0.0230, -0.0382,  ...,  0.0112,  0.0739,  0.0399],
        [-0.0075,  0.0569, -0.0903,  ..., -0.0105,  0.1205,  0.0362],
        ...,
        [-0.0320,  0.0622, -0.1089,  ...,  0.0226,  0.0922,  0.0389],
        [-0.0710,  0.0140, -0.0761,  ..., -0.0365,  0.0978,  0.0615],
        [-0.0357,  0.0151, -0.0686,  ..., -0.0102,  0.0744,  0.0069]],
       device='cuda:0', grad_fn=<AddmmBackward>)

In [179]:
xhat_delta

tensor([[-5.3743e-02,  4.4010e-02, -3.0419e-02,  ..., -3.1573e-02,
          3.3295e-02,  5.1158e-02],
        [-1.7625e-01,  1.2730e-01, -6.4805e-02,  ...,  5.5192e-02,
          1.4270e-01, -6.1634e-03],
        [-7.5377e-02, -1.6244e-02,  5.7163e-02,  ...,  5.2657e-02,
          6.2681e-02,  2.1037e-02],
        ...,
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [-1.9689e-01, -5.8483e-02,  9.4051e-03,  ...,  1.4330e-01,
          3.4291e-01,  1.2687e-01],
        [-8.1267e-03,  1.4335e-04, -5.4330e-04,  ..., -4.7268e-03,
          1.3943e-02, -8.6641e-03]], device='cuda:0', grad_fn=<SubBackward0>)

In [180]:
x_delta - xhat_delta

tensor([[ 0.0209,  0.0169, -0.0694,  ...,  0.0390,  0.0622, -0.0094],
        [ 0.0773, -0.1043,  0.0266,  ..., -0.0440, -0.0688,  0.0460],
        [ 0.0679,  0.0732, -0.1474,  ..., -0.0631,  0.0578,  0.0152],
        ...,
        [-0.0320,  0.0622, -0.1089,  ...,  0.0226,  0.0922,  0.0389],
        [ 0.1259,  0.0725, -0.0856,  ..., -0.1798, -0.2451, -0.0654],
        [-0.0276,  0.0149, -0.0681,  ..., -0.0054,  0.0605,  0.0156]],
       device='cuda:0', grad_fn=<SubBackward0>)

In [78]:
molopt.delta(x_embedding, y) - xhat

tensor([[ 0.0016, -0.0741, -0.0745,  ...,  0.0228, -0.0114, -0.0270],
        [ 0.0706, -0.2145, -0.2110,  ...,  0.3129,  0.1174, -0.0711],
        [-0.1300, -0.2984,  0.1859,  ..., -0.0819,  0.1519, -0.2849],
        ...,
        [ 0.2385, -0.1195, -0.1575,  ...,  0.0702, -0.1347,  0.0785],
        [ 0.0352,  0.1964, -0.0075,  ..., -0.0421,  0.1371, -0.1475],
        [-0.0727,  0.1660,  0.0481,  ...,  0.0867,  0.1587, -0.0863]],
       device='cuda:0', grad_fn=<SubBackward0>)

In [35]:
Y.mols[]